In [4]:
from loader import Loader, Loader2
import torch
import torchvision.transforms as transforms
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
import numpy as np
import model

In [3]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
latent_dim=32

In [4]:
def get_plabels_by_vae(vae, discriminator,samples,cycle):
    # sample1k = []
    sub5k = Loader2(is_train=False,  transform=transform_test, path_list=samples)
    ploader = torch.utils.data.DataLoader(sub5k, batch_size=1, shuffle=False, num_workers=2)

    unlabel_scores = []
    vae.eval()
    discriminator.eval()
    with torch.no_grad():
        for idx, (inputs, targets) in enumerate(ploader):
            inputs, targets = inputs.to(device), targets.to(device)
            _, _, mu, _ = vae(inputs)
            preds = discriminator(mu)
            # preds = preds.cpu().data
            unlabel_scores.append(preds.view(preds.size(0)).item())
            # progress_bar(idx, len(ploader))
    idx=np.argsort(unlabel_scores)
    samples=np.array(samples)
    return samples[idx[-1000:]]

In [5]:
vae = model.VAE(latent_dim)
discriminator = model.Discriminator(latent_dim)
vae=vae.to(device)
discriminator=discriminator.to(device)

/home/dang.hong.thanh/PT4AL/model.py:113: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


In [6]:
with open(f'./loss/batch_{1}.txt', 'r') as f:
    samples = f.readlines()

In [14]:
sub5k = Loader2(is_train=False,  transform=transform_test, path_list=samples)
ploader = torch.utils.data.DataLoader(sub5k, batch_size=1, shuffle=False, num_workers=2)

unlabel_scores = []
vae.eval()
discriminator.eval()
with torch.no_grad():
    for idx, (inputs, targets) in enumerate(ploader):
        inputs, targets = inputs.to(device), targets.to(device)
        _, _, mu, _ = vae(inputs)
        preds = discriminator(mu)
        # preds = preds.cpu().data
        print(preds)
        unlabel_scores.append(preds.view(preds.size(0)).item())
        # progress_bar(idx, len(ploader))

tensor([[0.5768]], device='cuda:0')
tensor([[0.5554]], device='cuda:0')
tensor([[0.5130]], device='cuda:0')
tensor([[0.3541]], device='cuda:0')
tensor([[0.5985]], device='cuda:0')
tensor([[0.4222]], device='cuda:0')
tensor([[0.5401]], device='cuda:0')
tensor([[0.6008]], device='cuda:0')
tensor([[0.4979]], device='cuda:0')
tensor([[0.6637]], device='cuda:0')
tensor([[0.4934]], device='cuda:0')
tensor([[0.6253]], device='cuda:0')
tensor([[0.6077]], device='cuda:0')
tensor([[0.4518]], device='cuda:0')
tensor([[0.4839]], device='cuda:0')
tensor([[0.5504]], device='cuda:0')
tensor([[0.5686]], device='cuda:0')
tensor([[0.4111]], device='cuda:0')
tensor([[0.5848]], device='cuda:0')
tensor([[0.6060]], device='cuda:0')
tensor([[0.6083]], device='cuda:0')
tensor([[0.6567]], device='cuda:0')
tensor([[0.3231]], device='cuda:0')
tensor([[0.5070]], device='cuda:0')
tensor([[0.5697]], device='cuda:0')
tensor([[0.5335]], device='cuda:0')
tensor([[0.4099]], device='cuda:0')
tensor([[0.4597]], device='c

KeyboardInterrupt: 

In [8]:
idx=np.argsort(unlabel_scores)
samples=np.array(samples)
samples[idx[-1000:]]

array(['./DATA/train/1/42221.png\n', './DATA/train/3/47830.png\n',
       './DATA/train/3/6462.png\n', './DATA/train/2/46584.png\n',
       './DATA/train/1/28326.png\n', './DATA/train/1/28326.png\n',
       './DATA/train/3/3905.png\n', './DATA/train/9/11227.png\n',
       './DATA/train/2/8543.png\n', './DATA/train/3/28574.png\n',
       './DATA/train/1/14470.png\n', './DATA/train/2/10412.png\n',
       './DATA/train/0/77.png\n', './DATA/train/4/17635.png\n',
       './DATA/train/8/40785.png\n', './DATA/train/2/5123.png\n',
       './DATA/train/3/35101.png\n', './DATA/train/6/16499.png\n',
       './DATA/train/7/36306.png\n', './DATA/train/2/46683.png\n',
       './DATA/train/3/33779.png\n', './DATA/train/6/17153.png\n',
       './DATA/train/7/46400.png\n', './DATA/train/3/3910.png\n',
       './DATA/train/1/45859.png\n', './DATA/train/6/34033.png\n',
       './DATA/train/9/24310.png\n', './DATA/train/4/28151.png\n',
       './DATA/train/9/43943.png\n', './DATA/train/6/29063.png\n',
   

In [20]:
np.sort(unlabel_scores)

array([0.15731855, 0.22305416, 0.24880323, 0.2780956 , 0.28975704,
       0.29166138, 0.29654577, 0.30668846, 0.30674157, 0.31836808,
       0.32154071, 0.32313526, 0.32401192, 0.32671022, 0.33060607,
       0.3317804 , 0.33636048, 0.33796883, 0.3391965 , 0.34037548,
       0.34688628, 0.34894875, 0.35073197, 0.35114604, 0.35408229,
       0.35779998, 0.35829973, 0.36137369, 0.36249787, 0.36588451,
       0.36894196, 0.37099043, 0.37154579, 0.37513202, 0.37645775,
       0.37682465, 0.37831241, 0.37945309, 0.38120455, 0.38209003,
       0.38566262, 0.38673076, 0.38702878, 0.39348251, 0.39482194,
       0.39604932, 0.40038046, 0.40052199, 0.40090019, 0.4010841 ,
       0.40314835, 0.40340772, 0.40376791, 0.40425554, 0.4053115 ,
       0.40594614, 0.40654755, 0.40723655, 0.40754098, 0.4098798 ,
       0.41047364, 0.41111693, 0.41198283, 0.41267821, 0.4135839 ,
       0.41475934, 0.41513443, 0.41740635, 0.42083094, 0.42086709,
       0.42216316, 0.423305  , 0.42342332, 0.42519224, 0.42568

: 

In [17]:
torch.topk(torch.from_numpy(np.array(unlabel_scores)),10)

torch.return_types.topk(
values=tensor([0.7517, 0.7493, 0.7462, 0.7326, 0.7281, 0.7205, 0.7197, 0.7158, 0.7074,
        0.7004], dtype=torch.float64),
indices=tensor([410, 386, 117, 194, 149, 208, 229, 215, 213, 417]))

In [18]:
np.sort(unlabel_scores)[-10:]

array([0.70044774, 0.70736086, 0.7158373 , 0.71974599, 0.72046977,
       0.72810984, 0.73260862, 0.74615794, 0.74929363, 0.75172842])

In [9]:
samples1k=get_plabels_by_vae(vae, discriminator,samples,cycle=2)

KeyboardInterrupt: 

In [1]:
from dataloaders.datasets.cityscapes import CityscapesSegmentation,ActiveCityscapesSegmentation


/home/dang.hong.thanh/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [3]:
cityspace_dir='/home/dang.hong.thanh/datasets/Cityspaces'
# image_transform=transforms.Compose([
#     # you can add other transformations in this list
#     transforms.ToTensor()])
# target_transform = transforms.Compose([
#     transforms.PILToTensor()
# ])
interp = nn.Upsample(size=(321, 321), mode='bilinear')


In [4]:
from models.deeplabv1 import DeepLabV1
net=DeepLabV1(n_classes=19, n_blocks=[3, 4, 23, 3])
net = net.to('cuda:0')

In [5]:
with open(f'./loss_seg/batch_0.txt', 'r') as f:
            samples = f.readlines()
samples=[sample[:-1] for sample in samples] # remove newline character

In [6]:
samples[0]

'/home/dang.hong.thanh/datasets/Cityspaces/images/train/ulm/ulm_000022_000019_leftImg8bit.png'

In [23]:
pth='/home/dang.hong.thanh/PT4AL/checkpoint/ckpt.pth'
checkpoint=torch.load(pth)
net.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [8]:
class Args:
    resize = 512
    base_size = 512
    crop_size = 321
    multi_scale= None

In [9]:
args=Args()

In [10]:
import numpy as np

In [ ]:
# def get_seg_plabels_confidence(net, samples, cycle):
    # dictionary with 10 keys as class labels
    # class_dict = {}
    # [class_dict.setdefault(x,[]) for x in range(10)]
device='cuda'
sample1k = []
subset = ActiveCityscapesSegmentation(args,cityspace_dir,split='train',files=samples)
# subset=CityscapesSegmentation(args,cityspace_dir,split='val')
ploader = torch.utils.data.DataLoader(subset, batch_size=1, shuffle=False, num_workers=2)

uncertainty_scores = []
net.eval()
with torch.no_grad():
    for idx, samples in enumerate(ploader):
        # print(inputs)
        # print(targets)
        inputs,targets=samples['image'],samples['label']
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = net(inputs)
        outputs=interp(outputs)
        # scores, predicted = outputs.max(1)
        # save top1 confidence score 
        outputs = F.normalize(outputs, dim=1)
        probs = F.softmax(outputs, dim=1)
        # top1_scores.append(probs[0][predicted.item()])
        # top1_scores.append(probs[0][predicted.item()].cpu().item())
        max_probs,indices=torch.max(probs,dim=1)
        # print(outputs.size())
        print(max_probs)
        # print(indices)
        measures=torch.logical_and(probs>=0.35,probs<=0.75)

        uncertainty_score=torch.sum(measures,dim=(1,2,3))
        # print(uncertainty_score)
        uncertainty_scores.append(uncertainty_score.cpu().item())
        # progress_bar(idx, len(ploader))
        if idx==10: break
idx = np.argsort(uncertainty_scores)
samples = np.array(samples)
    # return samples[idx[:100]]

In [ ]:
samples=subset[0]
inputs,targets=samples['image'],samples['label']
inputs, targets = inputs.to(device), targets.to(device)

In [24]:
outputs = net(inputs.unsqueeze(0))
outputs=interp(outputs)
scores, predicted = outputs.max(1)
print(scores)

tensor([[[5.1262, 5.1262, 5.1262,  ..., 3.5352, 3.5352, 3.5352],
         [5.1262, 5.1262, 5.1262,  ..., 3.5352, 3.5352, 3.5352],
         [5.1262, 5.1262, 5.1262,  ..., 3.5352, 3.5352, 3.5352],
         ...,
         [5.1479, 5.1479, 5.1479,  ..., 7.1912, 7.1912, 7.1912],
         [5.1479, 5.1479, 5.1479,  ..., 7.1912, 7.1912, 7.1912],
         [5.1479, 5.1479, 5.1479,  ..., 7.1912, 7.1912, 7.1912]]],
       device='cuda:0', grad_fn=<MaxBackward0>)


In [25]:
probs = F.softmax(outputs, dim=1)
print(probs)

tensor([[[[2.6195e-02, 2.6195e-02, 2.6195e-02,  ..., 7.8832e-02,
           7.8832e-02, 7.8832e-02],
          [2.6195e-02, 2.6195e-02, 2.6195e-02,  ..., 7.8832e-02,
           7.8832e-02, 7.8832e-02],
          [2.6195e-02, 2.6195e-02, 2.6195e-02,  ..., 7.8832e-02,
           7.8832e-02, 7.8832e-02],
          ...,
          [7.8099e-01, 7.8099e-01, 7.8099e-01,  ..., 9.7239e-01,
           9.7239e-01, 9.7239e-01],
          [7.8099e-01, 7.8099e-01, 7.8099e-01,  ..., 9.7239e-01,
           9.7239e-01, 9.7239e-01],
          [7.8099e-01, 7.8099e-01, 7.8099e-01,  ..., 9.7239e-01,
           9.7239e-01, 9.7239e-01]],

         [[4.3105e-03, 4.3105e-03, 4.3105e-03,  ..., 1.0714e-02,
           1.0714e-02, 1.0714e-02],
          [4.3105e-03, 4.3105e-03, 4.3105e-03,  ..., 1.0714e-02,
           1.0714e-02, 1.0714e-02],
          [4.3105e-03, 4.3105e-03, 4.3105e-03,  ..., 1.0714e-02,
           1.0714e-02, 1.0714e-02],
          ...,
          [6.2329e-02, 6.2329e-02, 6.2329e-02,  ..., 1.4504

In [20]:
n_outputs = F.normalize(outputs, dim=1)
scores, predicted = n_outputs.max(1)
print(scores)
probs = F.softmax(n_outputs, dim=1)
print(probs)

tensor([[[0.6502, 0.6502, 0.6502,  ..., 0.5187, 0.5187, 0.5187],
         [0.6502, 0.6502, 0.6502,  ..., 0.5187, 0.5187, 0.5187],
         [0.6502, 0.6502, 0.6502,  ..., 0.5187, 0.5187, 0.5187],
         ...,
         [0.6295, 0.6295, 0.6295,  ..., 0.7763, 0.7763, 0.7763],
         [0.6295, 0.6295, 0.6295,  ..., 0.7763, 0.7763, 0.7763],
         [0.6295, 0.6295, 0.6295,  ..., 0.7763, 0.7763, 0.7763]]],
       device='cuda:0', grad_fn=<MaxBackward0>)
tensor([[[[0.0631, 0.0631, 0.0631,  ..., 0.0662, 0.0662, 0.0662],
          [0.0631, 0.0631, 0.0631,  ..., 0.0662, 0.0662, 0.0662],
          [0.0631, 0.0631, 0.0631,  ..., 0.0662, 0.0662, 0.0662],
          ...,
          [0.0960, 0.0960, 0.0960,  ..., 0.1109, 0.1109, 0.1109],
          [0.0960, 0.0960, 0.0960,  ..., 0.1109, 0.1109, 0.1109],
          [0.0960, 0.0960, 0.0960,  ..., 0.1109, 0.1109, 0.1109]],

         [[0.0502, 0.0502, 0.0502,  ..., 0.0494, 0.0494, 0.0494],
          [0.0502, 0.0502, 0.0502,  ..., 0.0494, 0.0494, 0.0494],
 

In [26]:
max_probs,indices=torch.max(probs,dim=1)
# print(outputs.size())
print(max_probs)

tensor([[[0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
         [0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
         [0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
         ...,
         [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724],
         [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724],
         [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724]]],
       device='cuda:0', grad_fn=<MaxBackward0>)


In [27]:
max_probs,indices=torch.topk(probs,k=2,dim=1)
print(max_probs)

tensor([[[[0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
          [0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
          [0.8439, 0.8439, 0.8439,  ..., 0.4716, 0.4716, 0.4716],
          ...,
          [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724],
          [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724],
          [0.7810, 0.7810, 0.7810,  ..., 0.9724, 0.9724, 0.9724]],

         [[0.0537, 0.0537, 0.0537,  ..., 0.2308, 0.2308, 0.2308],
          [0.0537, 0.0537, 0.0537,  ..., 0.2308, 0.2308, 0.2308],
          [0.0537, 0.0537, 0.0537,  ..., 0.2308, 0.2308, 0.2308],
          ...,
          [0.0720, 0.0720, 0.0720,  ..., 0.0145, 0.0145, 0.0145],
          [0.0720, 0.0720, 0.0720,  ..., 0.0145, 0.0145, 0.0145],
          [0.0720, 0.0720, 0.0720,  ..., 0.0145, 0.0145, 0.0145]]]],
       device='cuda:0', grad_fn=<TopkBackward0>)


In [28]:
max_probs.shape

torch.Size([1, 2, 321, 321])

: 